In [13]:
import numpy as np
import sklearn 
from sklearn.metrics import confusion_matrix

import graphviz

from DecisionTreeClassifier import DecisionTreeClassifier

In [20]:
dt = DecisionTreeClassifier.DecisionTree()
credit = np.loadtxt('credit.txt', delimiter=',', skiprows=1)
print(credit.shape)

dt_credit = DecisionTreeClassifier.DecisionTree()
dt_credit.tree_grow(credit[:, :-1], credit[:, -1], nmin=1, minleaf=1, nfeat=None)
dt_credit.print_tree()
#dt_credit.plot_tree()

y_pred = dt_credit.tree_predict(credit[:, :-1])
y_true = credit[:,-1]
confusion_matrix(y_true, y_pred)

(10, 6)
 Node:  3 40.0
  Node:  0 45.0
   Leaf:  0.0
   Node:  1 1.0
    Leaf:  0.0
    Leaf:  1.0
  Leaf:  1.0


array([[5, 0],
       [0, 5]])

# PIMA Indians Dataset

In [19]:
pima = np.loadtxt('pima.txt', delimiter=',', skiprows=1)

dt_pima = DecisionTreeClassifier.DecisionTree()
dt_pima.tree_grow(pima[:, :-1], pima[:, -1], nmin=20, minleaf=5, nfeat=None)
#dt_pima.print_tree()
#dt_pima.plot_tree()

y_pred = dt_pima.tree_predict(pima[:, :-1])
y_true = pima[:,-1]
confusion_matrix(y_true, y_pred)

(767, 9)


array([[277, 223],
       [  0, 267]])